In [1]:
import requests, datetime, time
from pyquery import PyQuery as pq
from dataflows import Flow, printer, dump_to_path, sort_rows


def get_messages(before_id=None):
    url = 'https://t.me/s/MOHreport'
    if before_id:
        url += '?before=' + str(before_id)
    print('loading ' + url)
    for message in pq(requests.get(url).text)('[data-post]'):
        message_id = int(message.attrib['data-post'].replace('MOHreport/', ''))
        date_elts = message.find_class('tgme_widget_message_date')
        assert len(date_elts) == 1
        date_elt = date_elts[0]
        message_datetime = next(date_elt.iterchildren()).attrib['datetime']
        message_datetime = "".join(reversed("".join(reversed(message_datetime)).replace(':','',1)))
        message_datetime = datetime.datetime.strptime(message_datetime, '%Y-%m-%dT%H:%M:%S%z')
        content_elts = message.find_class('tgme_widget_message_bubble')
        assert len(content_elts) == 1
        content_elt = content_elts[0]
        message_htmls = []
        image_urls = []
        for child in content_elt.iterchildren():
            if 'tgme_widget_message_text' in list(child.classes):
                message_htmls.append(pq(child).html())
            elif 'tgme_widget_message_photo_wrap' in list(child.classes):
                image_urls.append(child.attrib['style'].split("url('")[1].split("'")[0])        
        message_html = "<br/><br/>".join(message_htmls)
        message_text = message_html.replace('<br/>', "\n")
        image_urls = ",".join(image_urls)
        yield {'id': message_id, 'date': message_datetime, 'text': message_text, 'images': image_urls}

        
def get_all_messages():
    last_message_id = None
    num_messages = 0
    while True:
        if num_messages > 0 and num_messages % 500 == 0: print('Loaded ' + str(num_messages) + ' messages..')
        if last_message_id and last_message_id <= 2525: break
        for message in get_messages(last_message_id):
            if not last_message_id or message['id'] < last_message_id:
                last_message_id = message['id']
            yield message
            num_messages += 1
        print('sleeping .1 seconds..')
        time.sleep(.1)


Flow(
    get_all_messages(),
    sort_rows('{date}', reverse=True),
    printer(tablefmt='html', num_rows=1),
    dump_to_path('data/MOHReport')
).process()

loading https://t.me/s/MOHreport
loading https://t.me/s/MOHreport?before=3141
loading https://t.me/s/MOHreport?before=3120
loading https://t.me/s/MOHreport?before=3100
loading https://t.me/s/MOHreport?before=3080
loading https://t.me/s/MOHreport?before=3060


loading https://t.me/s/MOHreport?before=3040
loading https://t.me/s/MOHreport?before=3020
loading https://t.me/s/MOHreport?before=2999
loading https://t.me/s/MOHreport?before=2979
loading https://t.me/s/MOHreport?before=2959
loading https://t.me/s/MOHreport?before=2938
loading https://t.me/s/MOHreport?before=2917
loading https://t.me/s/MOHreport?before=2897
loading https://t.me/s/MOHreport?before=2877
loading https://t.me/s/MOHreport?before=2857
loading https://t.me/s/MOHreport?before=2836
loading https://t.me/s/MOHreport?before=2816
loading https://t.me/s/MOHreport?before=2795
loading https://t.me/s/MOHreport?before=2775
loading https://t.me/s/MOHreport?before=2755
loading https://t.me/s/MOHreport?before=2734
loading https://t.me/s/MOHreport?before=2714
loading https://t.me/s/MOHreport?before=2694
loading https://t.me/s/MOHreport?before=2671
loading https://t.me/s/MOHreport?before=2651
loading https://t.me/s/MOHreport?before=2631
loading https://t.me/s/MOHreport?before=2611
loading ht

#,id (integer),date (datetime),text (string),images (string)
1,3160,2020-03-16 17:57:13,"מצ״ב מכתב ממנכל משרד הבריאות למנכ״ל הכנסת <i class=""emoji"" style=""background-image:url('//telegram.o ...",None
2,3159,2020-03-16 17:56:45,None,https://cdn4.telesco.pe/file/QwlQXvaJJyVAGtLNzT2cnvmAZ_PLl-GgKNJwljLx265D-C9lT3hTvQ8a6YoLeh2U1EXYDv8 ...
...,,,,
574,2511,2020-01-30 12:10:52,None,None


(<datapackage.package.Package at 0x7fc5d8620e48>,
 {'count_of_rows': 574,
  'bytes': 602603,
  'hash': '186d3abf7bdf6c8f45abe216b16f1f46',
  'dataset_name': None})

In [2]:
import os
CKAN_URL = 'https://www.odata.org.il'
if os.environ.get('CKAN_API_KEY'):
    CKAN_API_KEY = os.environ['CKAN_API_KEY']
else:
    import getpass
    CKAN_API_KEY = getpass.getpass('CKAN_API_KEY')

CKAN_API_KEY ····································


In [3]:
from dataflows import load
import json

data = Flow(
    load('data/MOHReport/datapackage.json')
).results()[0][0]

def format_row(row):
    row['date'] = row['date'].strftime('%Y-%m-%dT%H:%M:%S')
    row['images'] = '' if not row['images'] else row['images']
    return row

records = [format_row(row) for row in data]

print(records[0])

res = requests.post('https://www.odata.org.il/api/3/action/datastore_create', json={
    'resource_id': 'ce4c9482-cd3a-485b-af56-d3d7118a7552',
    'force': True,
    'primary_key': ['id'],
}, headers={'Authorization':CKAN_API_KEY})
print(res.status_code)
print(res.text)
assert res.status_code == 200

res = requests.post('https://www.odata.org.il/api/3/action/datastore_upsert', json={
    'resource_id': 'ce4c9482-cd3a-485b-af56-d3d7118a7552',
    'records': records,
    'method': 'upsert',
    'force': True
}, headers={'Authorization':CKAN_API_KEY})
print(res.status_code)
# print(res.text)
assert res.status_code == 200

{'id': 3160, 'date': '2020-03-16T17:57:13', 'text': 'מצ״ב מכתב ממנכל משרד הבריאות למנכ״ל הכנסת <i class="emoji" style="background-image:url(\'//telegram.org/img/emoji/40/E2989D.png\')"><b>☝️</b></i><i class="emoji" style="background-image:url(\'//telegram.org/img/emoji/40/E2989D.png\')"><b>☝️</b></i>', 'images': ''}
200
{"help": "https://www.odata.org.il/api/3/action/help_show?name=datastore_create", "success": true, "result": {"method": "insert", "primary_key": ["id"], "resource_id": "ce4c9482-cd3a-485b-af56-d3d7118a7552"}}
200


In [6]:
if os.environ.get('SERVICE_ACCOUNT_FILE'):
    SERVICE_ACCOUNT_FILE = os.environ['SERVICE_ACCOUNT_FILE']
else:
    import getpass
    SERVICE_ACCOUNT_FILE = getpass.getpass('SERVICE_ACCOUNT_FILE')

SERVICE_ACCOUNT_FILE ··························································


In [22]:
from google.oauth2 import service_account
import googleapiclient.discovery

SCOPES = ['https://www.googleapis.com/auth/drive']
credentials = service_account.Credentials.from_service_account_file(SERVICE_ACCOUNT_FILE, scopes=SCOPES)
spreadsheets = googleapiclient.discovery.build('sheets', 'v4', credentials=credentials).spreadsheets()

In [64]:
sheets_values = spreadsheets.values().get(spreadsheetId='19pKanFwuABaNyPGISihcqcPuFcIj5e3svcnr5LLm1ns', range='res_1!A:D').execute()['values']

In [65]:
sheets_data = {}
for rownum, row in enumerate(sheets_values):
    if rownum == 0: continue
    sheets_data[int(row[0])] = {
        'id': int(row[0]),
        'date': row[1],
        'text': row[2] if len(row) == 3 else '',
        'images': row[3] if len(row) == 4 else ''
    }

In [66]:
for row in sorted(data, key=lambda row: row['id']):
    if row['id'] not in sheets_data:
        value_input_option = 'RAW'
        insert_data_option = 'INSERT_ROWS'
        value_range_body = {
          "values": [[row['id'], row['date'], row['text'], row['images']]]
        }
        request = spreadsheets.values().append(spreadsheetId='19pKanFwuABaNyPGISihcqcPuFcIj5e3svcnr5LLm1ns', 
                                               range='res_1!A:D', 
                                               valueInputOption=value_input_option, 
                                               insertDataOption=insert_data_option, 
                                               body=value_range_body)
        request.execute()
        time.sleep(2)
